In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.datasets import fetch_openml
from sklearn.model_selection import cross_val_score
import pickle


In [32]:
data = pd.read_csv(r"C:\Users\shali\Desktop\DS_Road_Map\8. Machine Learning\Regression\Overfitting\Project\car-mpg.csv")
data

,mpg,cyl,disp,hp,wt,acc,yr,origin,car_type,car_name
0,18.0,8,307.0,130,3504,12.0,70,1,0,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,0,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,0,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,0,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,0,ford torino
...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1,1,ford mustang gl
394,44.0,4,97.0,52,2130,24.6,82,2,1,vw pickup
395,32.0,4,135.0,84,2295,11.6,82,1,1,dodge rampage
396,28.0,4,120.0,79,2625,18.6,82,1,1,ford ranger


In [33]:
data.head()

,mpg,cyl,disp,hp,wt,acc,yr,origin,car_type,car_name
0,18.0,8,307.0,130,3504,12.0,70,1,0,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,0,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,0,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,0,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,0,ford torino


In [34]:
data.columns

Index(['mpg', 'cyl', 'disp', 'hp', 'wt', 'acc', 'yr', 'origin', 'car_type',
       'car_name'],
      dtype='object')

In [35]:
data.describe

<bound method NDFrame.describe of       mpg  cyl   disp   hp    wt   acc  yr  origin  car_type  \
0    18.0    8  307.0  130  3504  12.0  70       1         0   
1    15.0    8  350.0  165  3693  11.5  70       1         0   
2    18.0    8  318.0  150  3436  11.0  70       1         0   
3    16.0    8  304.0  150  3433  12.0  70       1         0   
4    17.0    8  302.0  140  3449  10.5  70       1         0   
..    ...  ...    ...  ...   ...   ...  ..     ...       ...   
393  27.0    4  140.0   86  2790  15.6  82       1         1   
394  44.0    4   97.0   52  2130  24.6  82       2         1   
395  32.0    4  135.0   84  2295  11.6  82       1         1   
396  28.0    4  120.0   79  2625  18.6  82       1         1   
397  31.0    4  119.0   82  2720  19.4  82       1         1   

                      car_name  
0    chevrolet chevelle malibu  
1            buick skylark 320  
2           plymouth satellite  
3                amc rebel sst  
4                  ford torino  

In [36]:
data.index

RangeIndex(start=0, stop=398, step=1)

In [37]:
data.shape

(398, 10)

In [38]:
data = data.drop(["car_name"], axis=1)
data["origin"] = data["origin"].replace({1:'america',2:"europe",3:"asia"})
data = pd.get_dummies(data,columns =["origin"],dtype=int)
data = data.replace("?",np.nan)

In [39]:
data = data.apply(pd.to_numeric, errors ="ignore")
numeric_cols = data.select_dtypes(include=[np.number]).columns
data[numeric_cols] = data[numeric_cols].apply(lambda x: x.fillna(x.median()))

C:\Users\shali\AppData\Local\Temp\ipykernel_9280\4278785629.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = data.apply(pd.to_numeric, errors ="ignore")


In [40]:
data.head()

,mpg,cyl,disp,hp,wt,acc,yr,car_type,origin_america,origin_asia,origin_europe
0,18.0,8,307.0,130.0,3504,12.0,70,0,1,0,0
1,15.0,8,350.0,165.0,3693,11.5,70,0,1,0,0
2,18.0,8,318.0,150.0,3436,11.0,70,0,1,0,0
3,16.0,8,304.0,150.0,3433,12.0,70,0,1,0,0
4,17.0,8,302.0,140.0,3449,10.5,70,0,1,0,0


In [41]:
data.columns

Index(['mpg', 'cyl', 'disp', 'hp', 'wt', 'acc', 'yr', 'car_type',
       'origin_america', 'origin_asia', 'origin_europe'],
      dtype='object')

In [42]:
x = data.drop(["mpg"], axis=1)    #independent variable
y = data[["mpg"]]                 # dependent variable

In [43]:
#Scaling the data

x_s = preprocessing.scale(x)
x_s = pd.DataFrame(x_s, columns = x.columns) #converting scaled data into dataframe

y_s = preprocessing.scale(y)
y_s = pd.DataFrame(y_s, columns = y.columns) #ideally train, test data should be in columns

In [45]:
#Split into train, test set

x_train, X_test, y_train,y_test = train_test_split(x_s, y_s, test_size = 0.30, random_state = 1)
x_train.shape

(278, 10)